In [1]:
import requests
import pandas as pd
import yfinance as yf
from yahooquery import Ticker
import yahooquery as yq
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.ensemble import GradientBoostingRegressor

## importando pelo yahoo finance

# Todos os filtros para buscar reits a partir do YahooQuery

## Fundos imobiliarios

In [2]:
pd_fiis = pd.read_csv('statusinvest-busca-avancada.csv', sep=';')
len(pd_fiis['TICKER'])


579

## REITs

In [3]:
s = yq.Screener()

lista_screeners = ['reit_diversified',
    'reit_healthcare_facilities',
    'reit_hotel_motel',
    'reit_industrial',
    'reit_office',
   'reit_residential',
   'reit_retail']

lista_cod_reits = []
reits = s.get_screeners(lista_screeners, 100)
pd_reits = pd.DataFrame(reits)
json_reit_diversified = pd_reits.iloc[9]

for reit in json_reit_diversified:
    for item in reit:
        lista_cod_reits.append(item['symbol'])

len(lista_cod_reits)

189

# Realizando a requisição dos REITs

In [4]:
dataJson = list()

for Fii in pd_fiis['TICKER']:
    dataJson.append(Ticker(Fii + '.SA'))
len(dataJson)

Failed to obtain crumb.  Ability to retrieve data will be significantly limited.
Failed to obtain crumb.  Ability to retrieve data will be significantly limited.
Failed to obtain crumb.  Ability to retrieve data will be significantly limited.
Failed to obtain crumb.  Ability to retrieve data will be significantly limited.


KeyboardInterrupt: 

In [55]:
def build_dataset_stocks(all_stock_data):
    for data_stocks in dataJson:
        try:
            balance_sheet = data_stocks.all_financial_data()

            if isinstance(balance_sheet, str):
                continue

            start_date, end_date = balance_sheet['asOfDate'].iloc[[0, -1]]

            historic_value = data_stocks.history(start=start_date, end=end_date)

            balance_sheet = (
                balance_sheet
                .fillna(0)  # Substitui NaNs por 0
                .rename(columns={'asOfDate': 'date'})  # Renomeia a coluna
                .assign(date=lambda df: pd.to_datetime(df['date']),  # Converte para datetime
                        year=lambda df: df['date'].dt.year)  # Extrai o ano
            )

            historic_value = (
                historic_value
                .reset_index()  # Transforma índices em colunas para evitar problemas
                .assign(date=lambda df: pd.to_datetime(df['date']),  # Converte para datetime
                        year=lambda df: df['date'].dt.year)  # Extrai o ano
            )

            complete_values = pd.merge(balance_sheet, historic_value, on='year', how='inner')

            complete_values.drop(columns=['date_x'], inplace=True)  # Remove data duplicada
            complete_values.rename(columns={'date_y': 'date'}, inplace=True)

            complete_values['symbol'] = historic_value['symbol']

            complete_values['date'] = pd.to_datetime(complete_values['date'])
            complete_values.set_index('date', inplace=True)

            # Seleciona apenas as colunas numéricas para o resample
            numeric_columns = complete_values.select_dtypes(include=np.number)
            monthly_data = numeric_columns.resample('ME').mean()
            monthly_data.reset_index(inplace=True)

            all_stock_data.append(monthly_data)
        except Exception as e:
            print(f"Error processing data for {data_stocks}: {e}")

In [45]:
def buscar_serie(codigo):
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados?formato=json"
    response = requests.get(url)
    if response.status_code == 200:
        dados = response.json()
        df = pd.DataFrame(dados)
        if not df.empty:
            df['data'] = pd.to_datetime(df['data'], dayfirst=True)
            df['valor'] = df['valor'].astype(float)
            return df
        else:
            print(f"A série com código {codigo} retornou um DataFrame vazio.")
            return None
    else:
        print(f"Erro ao acessar a API para o código {codigo}: {response.status_code}")
        return None

def add_economic_indicators():
    # Dicionário com os códigos SGS dos índices econômicos
    indices_economicos = {
        "IPCA": 433,
        "INPC": 188,
        "IGPM": 189,
        "SELIC": 432,
        "PIB": 4380,
        "DESEMPREGO": 24369,
        "CAMBIO": 1,
        "RESERVAS": 13621,
    }

    # Buscando todos os índices e armazenando em um dicionário
    dados_indices = {}
    for nome, codigo in indices_economicos.items():
        try:
            df = buscar_serie(codigo)
            if df is not None:
                dados_indices[nome] = df
                print(f"Dados de {nome} carregados com sucesso!")
            else:
                print(f"Não foi possível carregar os dados de {nome}.")
        except Exception as e:
            print(f"Erro ao carregar os dados de {nome}: {e}")

    # Combinando todos os DataFrames em um único DataFrame
    df_final = None
    for nome, df in dados_indices.items():
        if df is not None:
            df = df.rename(columns={'valor': nome})  # Renomeia a coluna 'valor' para o nome do índice
            if df_final is None:
                df_final = df
            else:
                df_final = pd.merge(df_final, df, on='data', how='outer')  # Combina por data

    # Exibindo o DataFrame final
    if df_final is not None:
        print(df_final.head())
    else:
        print("Nenhum dado econômico foi carregado.")
    return df_final

In [47]:
df_indices = add_economic_indicators()
df_indices.rename(columns={'data': 'date'}, inplace=True)
df_indices

Dados de IPCA carregados com sucesso!
Dados de INPC carregados com sucesso!
Dados de IGPM carregados com sucesso!
Erro ao acessar a API para o código 432: 406
Não foi possível carregar os dados de SELIC.
Dados de PIB carregados com sucesso!
Dados de DESEMPREGO carregados com sucesso!
Erro ao acessar a API para o código 1: 406
Não foi possível carregar os dados de CAMBIO.
Erro ao acessar a API para o código 13621: 406
Não foi possível carregar os dados de RESERVAS.
        data  IPCA  INPC  IGPM  PIB  DESEMPREGO
0 1979-05-01   NaN  1.76   NaN  NaN         NaN
1 1979-06-01   NaN  3.00   NaN  NaN         NaN
2 1979-07-01   NaN  5.36   NaN  NaN         NaN
3 1979-08-01   NaN  5.79   NaN  NaN         NaN
4 1979-09-01   NaN  6.61   NaN  NaN         NaN


,date,IPCA,INPC,IGPM,PIB,DESEMPREGO
0,1979-05-01,NaN,1.76,NaN,NaN,NaN
1,1979-06-01,NaN,3.00,NaN,NaN,NaN
2,1979-07-01,NaN,5.36,NaN,NaN,NaN
3,1979-08-01,NaN,5.79,NaN,NaN,NaN
4,1979-09-01,NaN,6.61,NaN,NaN,NaN
...,...,...,...,...,...,...
547,2024-12-01,0.52,0.48,0.94,1003201.8,6.2
548,2025-01-01,0.16,0.00,0.27,951304.9,6.5
549,2025-02-01,1.31,1.48,1.06,977562.3,6.8
550,2025-03-01,0.56,0.51,-0.34,1062391.5,7.0


In [48]:
df_indices_cleaned = df_indices.dropna(axis=1, how='all')

In [49]:
df_indices_cleaned

,date,IPCA,INPC,IGPM,PIB,DESEMPREGO
0,1979-05-01,NaN,1.76,NaN,NaN,NaN
1,1979-06-01,NaN,3.00,NaN,NaN,NaN
2,1979-07-01,NaN,5.36,NaN,NaN,NaN
3,1979-08-01,NaN,5.79,NaN,NaN,NaN
4,1979-09-01,NaN,6.61,NaN,NaN,NaN
...,...,...,...,...,...,...
547,2024-12-01,0.52,0.48,0.94,1003201.8,6.2
548,2025-01-01,0.16,0.00,0.27,951304.9,6.5
549,2025-02-01,1.31,1.48,1.06,977562.3,6.8
550,2025-03-01,0.56,0.51,-0.34,1062391.5,7.0


In [51]:
df_indices_cleaned.set_index('date', inplace=True)
monthly_indices = df_indices_cleaned.resample('ME').mean()
monthly_indices

,IPCA,INPC,IGPM,PIB,DESEMPREGO
date,,,,,
1979-05-31,NaN,1.76,NaN,NaN,NaN
1979-06-30,NaN,3.00,NaN,NaN,NaN
1979-07-31,NaN,5.36,NaN,NaN,NaN
1979-08-31,NaN,5.79,NaN,NaN,NaN
1979-09-30,NaN,6.61,NaN,NaN,NaN
...,...,...,...,...,...
2024-12-31,0.52,0.48,0.94,1003201.8,6.2
2025-01-31,0.16,0.00,0.27,951304.9,6.5
2025-02-28,1.31,1.48,1.06,977562.3,6.8


In [56]:
all_stock_complete = []
build_dataset_stocks(all_stock_complete)

C:\Users\heito\anaconda3\envs\extract_data_B3\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
C:\Users\heito\anaconda3\envs\extract_data_B3\Lib\site-packages\yahooquery\ticker.py:1333: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["dividends"].fillna(0, inplace=True)
C:\Users\heito\anaconda3\envs\extract_data_B3\Lib\site-packages\yahooquery\utils

Error processing data for <yahooquery.ticker.Ticker object at 0x0000026739526E70>: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /ws/fundamentals-timeseries/v1/finance/timeseries/APXR11.SA?type=annualAmortization%2CannualAmortizationOfIntangiblesIncomeStatement%2CannualAverageDilutionEarnings%2CannualBasicAccountingChange%2CannualBasicAverageShares%2CannualBasicContinuousOperations%2CannualBasicDiscontinuousOperations%2CannualBasicEPS%2CannualBasicEPSOtherGainsLosses%2CannualBasicExtraordinary%2CannualContinuingAndDiscontinuedBasicEPS%2CannualContinuingAndDiscontinuedDilutedEPS%2CannualCostOfRevenue%2CannualDepletionIncomeStatement%2CannualDepreciationAmortizationDepletionIncomeStatement%2CannualDepreciationAndAmortizationInIncomeStatement%2CannualDepreciationIncomeStatement%2CannualDilutedAccountingChange%2CannualDilutedAverageShares%2CannualDilutedContinuousOperations%2CannualDilutedDiscontinuousOperations%2CannualDilutedEPS%2CannualDi

C:\Users\heito\anaconda3\envs\extract_data_B3\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
C:\Users\heito\anaconda3\envs\extract_data_B3\Lib\site-packages\yahooquery\ticker.py:1333: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["dividends"].fillna(0, inplace=True)


KeyboardInterrupt: 

In [10]:
len(all_stock_complete)

171

In [16]:
all_stock_complete[0]

,periodType,currencyCode,AccountsPayable,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,CapitalStock,CashAndCashEquivalents,...,WorkingCapital,year,symbol,open,high,low,close,volume,adjclose,dividends
date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,29410000.0,2021,ABCP11.SA,74.480003,74.480003,73.820000,74.370003,5057,49.654522,0.0
2021-01-05,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,29410000.0,2021,ABCP11.SA,74.370003,74.519997,73.849998,74.430000,3903,49.694565,0.0
2021-01-06,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,29410000.0,2021,ABCP11.SA,74.419998,74.419998,73.809998,73.959999,2177,49.380768,0.0
2021-01-07,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,29410000.0,2021,ABCP11.SA,73.949997,74.379997,73.949997,74.040001,1462,49.434185,0.0
2021-01-08,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,29410000.0,2021,ABCP11.SA,74.050003,74.389999,74.050003,74.269997,3397,49.587761,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-21,12M,BRL,54000.0,7306000.0,-3584000.0,4709082.0,16.46,8259000.0,59558000.0,7775000.0,...,12625000.0,2023,ABCP11.SA,68.199997,69.000000,68.050003,69.000000,904,53.761486,0.0
2023-12-22,12M,BRL,54000.0,7306000.0,-3584000.0,4709082.0,16.46,8259000.0,59558000.0,7775000.0,...,12625000.0,2023,ABCP11.SA,69.000000,70.000000,69.000000,69.500000,1408,54.151073,0.0
2023-12-26,12M,BRL,54000.0,7306000.0,-3584000.0,4709082.0,16.46,8259000.0,59558000.0,7775000.0,...,12625000.0,2023,ABCP11.SA,69.489998,70.040001,69.209999,69.830002,1188,54.408199,0.0


In [34]:
monthly_stocks = all_stock_complete

In [43]:
monthly_stocks[1]['date']

0     2021-06-30
1     2021-07-01
2     2021-07-02
3     2021-07-05
4     2021-07-06
         ...    
743   2024-06-24
744   2024-06-25
745   2024-06-26
746   2024-06-27
747   2024-06-28
Name: date, Length: 748, dtype: datetime64[ns]

In [13]:
df_stocks = pd.concat(all_stock_complete).drop_duplicates().reset_index(drop=True)
df_stocks = df_stocks.fillna(0)
df_stocks.to_csv('all_stocks.csv', index=False)

In [32]:
all_stocks = pd.read_csv('all_stocks.csv', sep=',')
all_stocks

,periodType,currencyCode,AccountsPayable,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,CapitalStock,CashAndCashEquivalents,...,InventoriesAdjustmentsAllowances,ImpairmentOfCapitalAssets,HedgingAssetsCurrent,CapitalLeaseObligations,CurrentCapitalLeaseObligation,LongTermCapitalLeaseObligation,ChangeInInventory,InvestmentsinJointVenturesatCost,ChangeInAccruedExpense,DividendPaidCFO
0,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125391,12M,BRL,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125392,12M,BRL,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125393,12M,BRL,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125394,12M,BRL,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
group = all_stocks.groupby('symbol')
group.groups

{'0': [28769, 28770, 28771, 28772, 28773, 28774, 28775, 28776, 28777, 28778, 28779, 28780, 28781, 28782, 28783, 28784, 28785, 28786, 28787, 28788, 28789, 28790, 28791, 28792, 28793, 28794, 28795, 28796, 28797, 28798, 28799, 28800, 28801, 28802, 28803, 28804, 28805, 28806, 28807, 28808, 28809, 28810, 28811, 28812, 28813, 28814, 28815, 28816, 28817, 28818, 28819, 28820, 28821, 28822, 28823, 28824, 28825, 28826, 28827, 28828, 28829, 28830, 28831, 28832, 28833, 28834, 28835, 28836, 28837, 28838, 28839, 28840, 28841, 28842, 28843, 28844, 28845, 28846, 28847, 28848, 28849, 28850, 28851, 28852, 28853, 28854, 28855, 28856, 28857, 28858, 28859, 28860, 28861, 28862, 28863, 28864, 28865, 28866, 28867, 28868, ...], 'ABCP11.SA': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,

In [36]:
all_stocks['date'] = pd.to_datetime(all_stocks['date'])

In [40]:
def resample_group(group, time_column, resample_rule):
    group = group.set_index(time_column)
    numeric_group = group.select_dtypes(include=['number'])
    return numeric_group.resample(resample_rule).mean()

def resample_groups(df, time_column, resample_rule):
    non_numeric_cols = df.select_dtypes(exclude=['number']).columns.drop([time_column, 'symbol'], errors='ignore')

    df = pd.get_dummies(df, columns=non_numeric_cols)

    def resample_group_wrapper(group):
        return resample_group(group, time_column, resample_rule)

    grouped = df.groupby('symbol')
    resampled_data = []
    for name, group in grouped:
        resampled_group = resample_group_wrapper(group)
        if resampled_group is not None and not resampled_group.empty:
            resampled_group['symbol'] = name  # Add the 'symbol' column back
            resampled_data.append(resampled_group)

    resampled_df = pd.concat(resampled_data).reset_index()
    return resampled_df

In [41]:
all_stocks_by_month = resample_groups(all_stocks, 'date', 'ME')
all_stocks_by_month

,date,AccountsPayable,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,CapitalStock,CashAndCashEquivalents,CashCashEquivalentsAndShortTermInvestments,...,ImpairmentOfCapitalAssets,HedgingAssetsCurrent,CapitalLeaseObligations,CurrentCapitalLeaseObligation,LongTermCapitalLeaseObligation,ChangeInInventory,InvestmentsinJointVenturesatCost,ChangeInAccruedExpense,DividendPaidCFO,symbol
0,2022-07-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2022-08-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2022-09-30,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2022-10-31,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2022-11-30,19141000.0,0.0,0.0,0.0,0.0,135160000.0,1.462259e+09,80000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,2023-08-31,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZIFI11.SA
6264,2023-09-30,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZIFI11.SA
6265,2023-10-31,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZIFI11.SA
6266,2023-11-30,0.0,0.0,0.0,0.0,0.0,377000.0,5.282700e+07,3000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ZIFI11.SA


In [42]:
all_stocks_by_month.to_csv('all_stocks_by_month.csv', index=False)

In [15]:
def merge_indices_stocks(all_stocks_param, df_indices_cleaned_param):
    # Verificar o formato das datas nos DataFrames
    all_stocks_param['date'] = pd.to_datetime(all_stocks_param['date'])
    df_indices_cleaned_param['date'] = pd.to_datetime(df_indices_cleaned_param['date'])

    # Verificar se há datas comuns entre os DataFrames
    common_dates = set(all_stocks_param['date']).intersection(set(df_indices_cleaned_param['date']))
    if not common_dates:
        print("Não há datas comuns entre os DataFrames.")
    else:
        # Realizar o merge dos DataFrames utilizando a coluna de datas
        merged_df = pd.merge(all_stocks_param, df_indices_cleaned_param, on='date', how='inner')

        # Exibir o DataFrame resultante
        return merged_df

In [16]:
all_stocks = merge_indices_stocks(all_stocks, df_indices_cleaned)

In [17]:
all_stocks

,periodType,currencyCode,AccountsPayable,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,CapitalStock,CashAndCashEquivalents,...,ChangeInAccruedExpense,DividendPaidCFO,IPCA,INPC,IGPM,SELIC,PIB,DESEMPREGO,CAMBIO,RESERVAS
0,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,0.0,0.0,NaN,NaN,NaN,2.00,NaN,NaN,5.1626,355760.0
1,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,0.0,0.0,NaN,NaN,NaN,2.00,NaN,NaN,5.3269,355847.0
2,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,0.0,0.0,NaN,NaN,NaN,2.00,NaN,NaN,5.3182,355515.0
3,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,0.0,0.0,NaN,NaN,NaN,2.00,NaN,NaN,5.3433,355437.0
4,12M,BRL,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,...,0.0,0.0,NaN,NaN,NaN,2.00,NaN,NaN,5.3683,355244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125391,12M,BRL,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,...,0.0,0.0,NaN,NaN,NaN,11.75,NaN,NaN,4.8663,353248.0
125392,12M,BRL,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,...,0.0,0.0,NaN,NaN,NaN,11.75,NaN,NaN,4.8766,353406.0
125393,12M,BRL,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,...,0.0,0.0,NaN,NaN,NaN,11.75,NaN,NaN,4.8619,354174.0
125394,12M,BRL,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,...,0.0,0.0,NaN,NaN,NaN,11.75,NaN,NaN,4.8368,354177.0


In [18]:
def remover_colunas_correlacionadas(df):
    matrix_corr = df.corr().abs()
    remove_columns = set()

    for i_corr in range(len(matrix_corr.columns)):
        for j_corr in range(i_corr + 1, len(matrix_corr.columns)):
            if matrix_corr.iloc[i_corr, j_corr] > 0.87 and matrix_corr.columns[j_corr] != 'close':
                remove_columns.add(matrix_corr.columns[j_corr])

    return df.drop(columns = remove_columns)

def filtra_colunas_zero(df):
    mean = df.describe().loc['mean']
    cols_to_keep = mean[mean != 0].index
    return df[cols_to_keep]

def remover_colunas_nao_numericas(df):
    return df.drop(['currencyCode', 'periodType'], axis=1)

In [19]:
df_fil = (all_stocks.pipe(remover_colunas_nao_numericas)
               .pipe(filtra_colunas_zero)
               .pipe(remover_colunas_correlacionadas)
               )
df_fil

,AccountsPayable,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,CapitalStock,CashAndCashEquivalents,CashCashEquivalentsAndShortTermInvestments,CashDividendsPaid,...,CapitalLeaseObligations,ChangeInInventory,InvestmentsinJointVenturesatCost,ChangeInAccruedExpense,DividendPaidCFO,IPCA,IGPM,SELIC,CAMBIO,RESERVAS
0,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,17635000.0,-57471000.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2.00,5.1626,355760.0
1,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,17635000.0,-57471000.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2.00,5.3269,355847.0
2,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,17635000.0,-57471000.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2.00,5.3182,355515.0
3,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,17635000.0,-57471000.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2.00,5.3433,355437.0
4,0.0,19715000.0,-10260000.0,12203833.0,8.64,12616000.0,154347000.0,17635000.0,17635000.0,-57471000.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2.00,5.3683,355244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125391,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,11.75,4.8663,353248.0
125392,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,11.75,4.8766,353406.0
125393,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,11.75,4.8619,354174.0
125394,0.0,0.0,0.0,0.0,0.00,377000.0,52827000.0,3000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,11.75,4.8368,354177.0


In [20]:
print(f'quantidade colunas antes filtro: {len(all_stocks.columns)} || quantidade colunas após filtro: {len(df_fil.columns)}')

quantidade colunas antes filtro: 252 || quantidade colunas após filtro: 154


In [21]:
#df_fil = df_fil.dropna()
df_fil = df_fil.fillna(0)
df_fil['open']

0          74.480003
1          74.370003
2          74.419998
3          73.949997
4          74.050003
             ...    
125391    800.000000
125392    810.000000
125393    810.000000
125394    849.000000
125395    849.000000
Name: open, Length: 125396, dtype: float64

# Filtrar todos os fundos que o request na API conseguiu ter acesso aos dados:

### Exemplo de exibição, simpleInfo é uma lista de dataframes

## Filtrar com base na media de volume - Minimo: 1000

In [22]:
info_target = df_fil[['open', 'close']]
info_target

,open,close
0,74.480003,74.370003
1,74.370003,74.430000
2,74.419998,73.959999
3,73.949997,74.040001
4,74.050003,74.269997
...,...,...
125391,800.000000,810.000000
125392,810.000000,810.000000
125393,810.000000,810.000000
125394,849.000000,849.000000


## Adicionar colunas com a regressão linear, comparando com o valor: b3, IPCA, tesouro direto

ibova = pd.DataFrame(yf.Ticker('BOVA11.SA').history(period="max"))
ibova.index = ibova.index.strftime('%d/%m/%Y')
ibova

In [23]:
df_fil.drop(columns=['open', 'close'], inplace=True)

In [24]:
dados1 = np.array(df_fil)
dados2 = np.array(info_target['close'])

X_train, X_test, y_train, y_test = train_test_split(dados1, dados2, test_size=0.2, random_state=42)
# Criar e treinar o modelo
# Criando o modelo Random Forest
# Create and train the model
model_regression = GradientBoostingRegressor(n_estimators=100, random_state=42)
model_regression.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model_regression.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Criar um DataFrame com os valores reais e previstos
resultados = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R² Score: {r2:.4f}")

Mean Absolute Error (MAE): 8.1087
Mean Squared Error (MSE): 254.3111
R² Score: 1.0000


In [25]:
resultados

,Actual,Predicted
0,85.500000,80.152066
1,950.010010,993.956698
2,102.846260,99.603757
3,45.009998,46.988541
4,98.415642,101.057914
...,...,...
25075,96.949997,87.522345
25076,11.200000,15.715841
25077,165.000000,156.276840
25078,93.250000,92.566390


In [26]:

# Supondo que 'df_fil' é o seu DataFrame de características
dados = np.array(df_fil)

# Dividir os dados em conjuntos de treino e teste
X_train, X_test = train_test_split(dados, test_size=0.2, random_state=42)

# Definir o número de clusters
num_clusters = 3  # Você pode ajustar este número conforme necessário

# Criar e treinar o modelo K-Means
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X_train)

# Prever os clusters para o conjunto de teste
clusters = kmeans.predict(X_test)

# Avaliar o modelo usando a pontuação de silhueta
silhouette_avg = silhouette_score(X_test, clusters)
print(f"Silhouette Score: {silhouette_avg:.4f}")

# Criar um DataFrame com os clusters previstos
resultados = pd.DataFrame(X_test, columns=df_fil.columns)
resultados['Cluster'] = clusters


Silhouette Score: 0.9964


In [27]:
# Exibir os primeiros resultados
resultados[(resultados['Cluster'] == 2)]

,AccountsPayable,AccountsReceivable,AllowanceForDoubtfulAccountsReceivable,BasicAverageShares,BasicEPS,BeginningCashPosition,CapitalStock,CashAndCashEquivalents,CashCashEquivalentsAndShortTermInvestments,CashDividendsPaid,...,ChangeInInventory,InvestmentsinJointVenturesatCost,ChangeInAccruedExpense,DividendPaidCFO,IPCA,IGPM,SELIC,CAMBIO,RESERVAS,Cluster
586,0.0,0.0,0.0,0.0,0.0,4506000.0,199842000.0,2422000.0,2422000.0,-20763000.0,...,0.0,0.0,0.0,0.0,0.00,0.00,13.75,4.9292,344471.0,2
1086,0.0,0.0,0.0,0.0,0.0,4506000.0,199842000.0,2422000.0,2422000.0,-20763000.0,...,0.0,0.0,0.0,0.0,0.00,0.00,13.75,4.9513,345875.0,2
1200,0.0,0.0,0.0,0.0,0.0,4506000.0,199842000.0,2422000.0,2422000.0,-20763000.0,...,0.0,0.0,0.0,0.0,0.00,0.00,12.75,5.1918,338888.0,2
1365,0.0,0.0,0.0,0.0,0.0,4506000.0,199842000.0,2422000.0,2422000.0,-20763000.0,...,0.0,0.0,0.0,0.0,0.00,0.00,13.25,4.8706,343332.0,2
2128,0.0,0.0,0.0,0.0,0.0,4506000.0,199842000.0,2422000.0,2422000.0,-20763000.0,...,0.0,0.0,0.0,0.0,0.00,0.00,13.75,5.1695,331805.0,2
2821,0.0,0.0,0.0,0.0,0.0,4506000.0,199842000.0,2422000.0,2422000.0,-20763000.0,...,0.0,0.0,0.0,0.0,0.00,0.00,12.25,4.8931,346157.0,2
2906,0.0,0.0,0.0,0.0,0.0,4506000.0,199842000.0,2422000.0,2422000.0,-20763000.0,...,0.0,0.0,0.0,0.0,0.00,0.00,13.75,4.8038,346776.0,2
3153,0.0,0.0,0.0,0.0,0.0,4506000.0,199842000.0,2422000.0,2422000.0,-20763000.0,...,0.0,0.0,0.0,0.0,0.00,0.00,13.75,4.7698,344904.0,2
3535,0.0,0.0,0.0,0.0,0.0,4506000.0,199842000.0,2422000.0,2422000.0,-20763000.0,...,0.0,0.0,0.0,0.0,0.00,0.00,11.75,4.8766,353406.0,2
4194,0.0,0.0,0.0,0.0,0.0,4506000.0,199842000.0,2422000.0,2422000.0,-20763000.0,...,0.0,0.0,0.0,0.0,0.00,0.00,13.75,5.2395,328166.0,2
